<a href="https://colab.research.google.com/github/fregean/python3basic/blob/master/Python3notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python３エンジニア認定基礎試験講座

###（体験クラス用 **60分**）

<a href="https://diveintocode.gyazo.com/8adeb2476d22b1e9f7bd5909ea4acd95"><img src="https://t.gyazo.com/teams/diveintocode/8adeb2476d22b1e9f7bd5909ea4acd95.png" alt="Image from Gyazo" width="８00"/></a>

## １. **DIVE INTO EXAM** の問題を**５問**解きましょう（制限時間１分 × ５問）

<a href="https://diveintocode.gyazo.com/0260a79a95690af701391b444c2fa2ec"><img src="https://t.gyazo.com/teams/diveintocode/0260a79a95690af701391b444c2fa2ec.png" alt="Image from Gyazo" width="300"/></a>




以下のリンクから受験ページへアクセスしてください。



https://staging-exam.diveintocode.jp/exam/exam_categories/19/result_lists


<a href="https://diveintocode.gyazo.com/bfb8976b0fa474d1bf08fe72876cb6ac"><img src="https://t.gyazo.com/teams/diveintocode/bfb8976b0fa474d1bf08fe72876cb6ac.png" alt="Image from Gyazo" width="2334"/></a>



### 模擬試験お疲れさまでした！！

---

### 休憩時間（3分）

ここで、みなさんがこれから利用するColaboratoryというサービスについて説明します。

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Colaboratory とは</h1>

Colaboratory（略称: Colab）とは、ブラウザから Python を記述し実行できる環境のことを指します。

Colab は、<strong>学生</strong>、<strong>データ サイエンティスト</strong>、<strong>AI リサーチャー</strong>の作業を効率化するツールです。詳しくは、<a href="https://www.youtube.com/watch?v=inN8seMm7UI">Colab のご紹介</a>をご覧ください。

### <strong>このページは何？</strong>

このページは、静的なウェブページではなく、<strong>Colab ノートブック</strong>という、コードを記述して実行できる環境です。

たとえば次の<strong>コードセル</strong>に値を入力して実行することができます。実行後は結果の出力が表示されます。

## ２. 動画を見てみましょう（制限時間10分）

このあとの「動画の要約を話してみよう」の時間に一人ずつ要約を語ってもらいます。
動画は、内容を要約する気持ちで視聴してくださいね。

それでは、以下のページから動画にアクセスしてください。



今回の動画で取り扱う内容は、
- 数値
- 文字列
- リスト

です。



<a href="https://diveintocode.gyazo.com/1990623f930945b6776f65b318b56588"><img src="https://t.gyazo.com/teams/diveintocode/1990623f930945b6776f65b318b56588.png" alt="Image from Gyazo" width="300"/></a>

---

### 動画視聴お疲れさまでした！！

---

### 休憩時間（3分）

## ３. みんなに動画の要約を話してみよう（制限時間10分）


先ほど視聴した動画の要約をまわりの人に伝えましょう。

<a href="https://diveintocode.gyazo.com/5f613bec1b713b417b6665cbf5305c93"><img src="https://t.gyazo.com/teams/diveintocode/5f613bec1b713b417b6665cbf5305c93.png" alt="Image from Gyazo" width="300"/></a>

#### ★メモ帳★

まわりの人の要約を聞いて、感じたことをここにメモしましょう。

（例） 












### 話し合いお疲れさまでした！！（^0^）

---

### 休憩時間（3分）

## ４. 探求してみよう（制限時間15分）

ここまでの30分間に、みなさんは模擬試験を受けて、動画を視聴し、その内容の要約を人に話したり聞いたりしました。

いまみなさんの頭の中には、「数値」と「文字列」と「リスト」についての、さまざまな情報や、それに結びついたイメージがいり混じっていると思います。

ここからの15分は、ひとりでその情報やイメージの整理を行ってください。

15分たったら休憩をとって、再びDIVE INTO EXAMの模擬試験を5分間解いてもらいます。



<a href="https://diveintocode.gyazo.com/b7cae49ff0042d1b430e751c1c762773"><img src="https://t.gyazo.com/teams/diveintocode/b7cae49ff0042d1b430e751c1c762773.png" alt="Image from Gyazo" width="300"/></a>

**Python3エンジニア認定基礎試験**の 出題範囲は、Python公式サイトでも公開されています。

今回の講座で扱っている範囲の内容は、以下のリンク先のページに記載されています。

[数](https://docs.python.org/ja/3.5/tutorial/introduction.html#numbers)

[文字列型](https://docs.python.org/ja/3.5/tutorial/introduction.html#strings)

[リスト型](https://docs.python.org/ja/3.5/tutorial/introduction.html#lists)



<a href="https://diveintocode.gyazo.com/74b90899106e751c12a1e391e7114860"><img src="https://t.gyazo.com/teams/diveintocode/74b90899106e751c12a1e391e7114860.png" alt="Image from Gyazo" width="300"/></a>

>## 数値
<strong>定数</strong>と<strong>変数</strong>

- 定数を計算する
  - 数字の並びを作る
  - 数式を作る
  - int型（整数型）とfloat型（浮動小数点数型）を使う
  - 操作の順序を制御する
- 変数を作る
  - 変数に代入する
- ほかの<strong>数値型</strong> を知る

<a href="https://diveintocode.gyazo.com/dd25c3a16420f09bdaafd63c443d368d"><img src="https://t.gyazo.com/teams/diveintocode/dd25c3a16420f09bdaafd63c443d368d.png" alt="Image from Gyazo" width="300"/></a>

---

>## 文字列
基本の<strong>シーケンス</strong>

- 文字列を作る
  - 引用符（クォート）
  - 連結・繰り返し
- エスケープ（文字の意味を無効化する）記法を使う
  - print()関数
  - 改行する
  - エスケープ記法を無効にする
- 変数に代入する
- 位置を<strong>インデックス</strong>で指定する（インデクシング）
- <strong>スライス</strong>で部分的に切り取る（スライシング）
- <strong>変更不可能</strong>である（イミュータブル）
  - シーケンスの性質


<br>
文字列の作成には引用符を用い、それらを開閉する必要があります。

<a href="https://diveintocode.gyazo.com/fe984dc42dcfebadbaaba011e0c3ce18"><img src="https://t.gyazo.com/teams/diveintocode/fe984dc42dcfebadbaaba011e0c3ce18.png" alt="Image from Gyazo" width="350"/></a>

---

>## リスト
複雑な構造をもつ<strong>シーケンス</strong>

- リストを作る
  - 角カッコ
  - カンマ区切りの値（要素）
  - 連結・繰り返し
  - 異なる型の要素を入れる
- 変数に代入する
- 位置を<strong>インデックス</strong>で指定する（インデクシング）
- <strong>スライス</strong>で部分的に切り取る（スライシング）
- <strong><font color="FireBrick">変更可能</font></strong>である（ミュータブル）
  - シーケンスの性質
- 末尾に要素を追加する<strong>append()</strong>メソッド
- <strong>len()</strong>関数でリストの要素数を返す
- リストのリスト

### 探求お疲れさまでした！！ （^▽^）

---

### 休憩時間（3分）

## ５. **DIVE INTO EXAM** の問題を**５問**解きましょう（制限時間１分 × ５問）

以下のリンクから受験ページへアクセスしてください。


https://staging-exam.diveintocode.jp/exam/exam_categories/19/result_lists

---

<a href="https://diveintocode.gyazo.com/0260a79a95690af701391b444c2fa2ec"><img src="https://t.gyazo.com/teams/diveintocode/0260a79a95690af701391b444c2fa2ec.png" alt="Image from Gyazo" width="300"/></a>

### 講座の受講お疲れさまでした！！

### ３分間休憩しましょう！